
<br> <br> <br> <br>
<br> <br> <br> <br>

<center>


### EAI 6000 Fundamentals of AI
### Assignment 2 Regression Models
### Shivangi Vashi
### Anirudh Sharma
### Sandeep Srivastava
### Sweta Mankala


<br> <br> <br> <br>
<br> <br> <br> <br>
  

### Northeastern University
### Instructor: Kasun Samarasinghe
### Fall 2020
### 8 November 2020
</center>

******
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>
<br> <br> <br> <br>

<style>
body {
text-align: justify
font-size: 16pt}
</style>

<center>
## Introduction  </center>

<br><br>
Data is from: [policing data](https://openpolicing.stanford.edu/data/).
<br>
Some of questions we explore are as follows-

1. How are traffic stops distributed among race?<br>
2. How has this changed over the years?<br>
3. Who violates the law more often? Men/Women , on the basis of race as well<br>

We primarily look at traffic stops data from Massachusetts. The stops can be pedestrian or vehicular, however for MA only vehicular stop data is present. We chose this data because this is a very relevant problem currently, and this might help us understand quantitatively whether a racial bias exist.

We might further expand our analyses in the future by considering data from multiple states or all of USA to compare our results.
<br><br>

This week we are tasked with exploring Linear and Logistic Regression models to predict an outcome. Based on our initial Exploratory analysis, we will perform feature extraction and analysis to determine what relevant features can be used to build our model.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import tensorflow as tf

In [3]:
#Importing data
openpolicing_path="../ma_statewide_2020_04_01.csv"

data=pd.read_csv(openpolicing_path,low_memory=False)

In [4]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


We check the data types to see if appropriate types are being used. We will convert them accordinly.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   raw_row_number              int64  
 1   date                        object 
 2   location                    object 
 3   county_name                 object 
 4   subject_age                 float64
 5   subject_race                object 
 6   subject_sex                 object 
 7   type                        object 
 8   arrest_made                 object 
 9   citation_issued             object 
 10  warning_issued              object 
 11  outcome                     object 
 12  contraband_found            object 
 13  contraband_drugs            object 
 14  contraband_weapons          object 
 15  contraband_alcohol          bool   
 16  contraband_other            object 
 17  frisk_performed             object 
 18  search_conducted            bool   
 19  search_basis         

We convert the date from object to date type and extract time for future analyses.

In [6]:
##Convert date to datetype
data.date
data["date"] = pd.to_datetime(data.date, format="%Y-%m-%d")
#Extract time
# data['time'] = [d.time() for d in data['date']]

In [7]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


In [8]:
#Convering race, sex, location etc to categorical
data.iloc[:,np.r_[2:4,5:8,9:12,19:23]]=data.iloc[:,np.r_[2:4,5:8,9:12,19:23]].astype("category")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   raw_row_number              int64         
 1   date                        datetime64[ns]
 2   location                    category      
 3   county_name                 category      
 4   subject_age                 float64       
 5   subject_race                category      
 6   subject_sex                 category      
 7   type                        category      
 8   arrest_made                 object        
 9   citation_issued             category      
 10  warning_issued              category      
 11  outcome                     category      
 12  contraband_found            object        
 13  contraband_drugs            object        
 14  contraband_weapons          object        
 15  contraband_alcohol          bool          
 16  contraband_other  

In [10]:
#Checking proportion of missing values
data.isnull().sum()*100/len(data)

raw_row_number                 0.000000
date                           0.000000
location                       0.195127
county_name                    0.195127
subject_age                    4.625146
subject_race                   0.048709
subject_sex                    0.457316
type                           0.000000
arrest_made                    0.026813
citation_issued                0.026813
warning_issued                 0.026813
outcome                        0.200747
contraband_found              98.368673
contraband_drugs              98.368673
contraband_weapons            98.368673
contraband_alcohol             0.000000
contraband_other              98.368673
frisk_performed               98.400843
search_conducted               0.000000
search_basis                  98.506281
reason_for_stop               48.579432
vehicle_type                   0.145277
vehicle_registration_state     0.287275
raw_Race                       0.048709
dtype: float64

<!-- We will use the KNN Imputer for imputing values into the Age and Sex variables for better precision -->

In [13]:
#Removing null, and unknown values from race
data=data[-data["subject_race"].isin(["other","unknown",np.nan])]
data["subject_race"].unique()

[white, hispanic, black, asian/pacific islander]
Categories (4, object): [white, hispanic, black, asian/pacific islander]

In [ ]:
# Impute the missing values.
# from sklearn.impute import SimpleImputer

# # impute missing data
# my_imputer = SimpleImputer(strategy='most_frequent')
# data["subject_"] = pd.DataFrame(my_imputer.fit_transform(data["subject_"]))

Encoding categorical variables for future analyses

In [ ]:
#### Below code doesnt work Error:Input contains NaN

In [ ]:
## One hot encoding race and other categorical variables
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(categories='auto')
# df = pd.DataFrame(data)
# feature_arr = ohe.fit_transform(df[['subject_race','arrest_made','citation_issued','outcome','contraband_found','contraband_drugs','contraband_weapons','contraband_alcohol','contraband_other']]).toarray()
# feature_labels = ohe.categories_
# feature_labels = np.array(feature_labels).ravel()

# features = pd.DataFrame(feature_arr, columns=feature_labels)

We use the get_dummies function to convert categorical variables to one hot encoded indicator variables

In [28]:
#One hot encoding categorical variables

df=pd.get_dummies(data.loc[:,['subject_race','subject_sex','arrest_made','citation_issued','outcome','contraband_found','contraband_drugs','contraband_weapons','contraband_alcohol','contraband_other','search_basis','reason_for_stop']])

In [51]:
data_encoded=pd.concat([data,df],axis=1)

We create a new binomial variable that tells us if the stop was before or after sunset

In [56]:
# from suntime import Sun, SunTimeException

# latitude = 42.407211
# longitude = -71.382439

# sun = Sun(latitude, longitude)

# # Get sunrise and sunset in UTC
# # sunrise=[]
# sunset=[]
# for value in data["date"]:
# #     sunrise.append(sun.get_sunrise_time(value))
#     sunset.append(sun.get_sunset_time(value))
#     format(value.strftime('%H:%M:%S'))

In [69]:
# sunset=pd.to_datetime(sunset)
# data["time"]=pd.to_datetime(data["time"])

In [ ]:
# Creating binomial variable for whether the stop was before or after sunset
# is_sunset=[]
# for i in range(len(sunset)):
#     for value in data["time"]:
#         data["is_sunset"]=np.where(value.time()>sunset[i].time(),"True","False")


In [105]:
# group_agg = data.groupby(["time"]).agg({
#   "search_conducted" : lambda x: x.mean(skipna = True), 
#   "frisk_performed" : lambda x: x.mean(skipna = True), 
#   "contraband_found" : lambda x: x.mean(skipna = True)
#   })
# group_agg.rename(columns={"search_conducted" : 'search_rate', "frisk_performed" : 'frisk_rate',"contraband_found":'success_rate'}, inplace=True)

# print(len(group_agg))

### Logistic Regression Model

We use the encoded and processed features and perform the following steps:
-Normalize the data
-Principle Component Analysis
-Split the data into train and test sets
-Train the model
-Test it on the test split
-Optimize via regularization

In [52]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3386549 entries, 0 to 3416237
Data columns (total 58 columns):
 #   Column                                    Dtype         
---  ------                                    -----         
 0   raw_row_number                            int64         
 1   date                                      datetime64[ns]
 2   location                                  category      
 3   county_name                               category      
 4   subject_age                               float64       
 5   subject_race                              category      
 6   subject_sex                               category      
 7   type                                      category      
 8   arrest_made                               object        
 9   citation_issued                           category      
 10  warning_issued                            category      
 11  outcome                                   category      
 12  contraband_fou

In [ ]:
data_encoded.drop(['subject_race','subject_sex','citation_issued','outcome','contraband_found','search_basis','reason_for_stop'],axis=1, inplace = True)

data_encoded.head()

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaled_features=data_encoded.copy()
# scale_cols=scaled_features.iloc[:,np.r_[4,17:19,25:57]]

# scaler = StandardScaler().fit(scale_cols.values)
# features = scaler.transform(scale_cols.values)

# scaled_features[scale_cols] = features
# print(scaled_features)

In [ ]:
# feat_cols = ['feature'+str(i) for i in range(data_encoded.shape[1])]
# normalised_data = pd.DataFrame(data_encoded,columns=feat_cols)
# normalised_data.tail()

In [ ]:
# PCA
# from sklearn.decomposition import PCA
# pca_data = PCA(n_components=20)
# principalComponents = pca_data.fit_transform(data_encoded)

In [60]:
data_encoded.head()

,raw_row_number,date,location,county_name,subject_age,type,arrest_made,warning_issued,contraband_drugs,contraband_weapons,...,search_basis_consent,search_basis_other,search_basis_probable cause,reason_for_stop_ChildRest,reason_for_stop_SeatBelt,"reason_for_stop_SeatBelt,ChildRest",reason_for_stop_Speed,"reason_for_stop_Speed,ChildRest","reason_for_stop_Speed,SeatBelt","reason_for_stop_Speed,SeatBelt,ChildRest"
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,vehicular,False,False,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
1,2,2007-06-07,SEEKONK,Bristol County,36.0,vehicular,False,True,False,False,...,0,1,0,0,0,0,0,0,0,0
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,vehicular,False,True,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,vehicular,False,True,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,5,2007-06-07,EVERETT,Middlesex County,22.0,vehicular,False,False,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


from sklearn.model_selection import train_test_split

X = data_encoded.drop(['subject_race'], axis=1)

y = data_encoded['subject_race']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

### Model Training

In [ ]:
# instantiate the model
logreg = LogisticRegression(C=100,solver='liblinear', random_state=0)


# fit the model
logreg.fit(X_train, y_train)

### Model Testing

In [ ]:
y_pred_test = logreg.predict(X_test)

y_pred_test

logreg.predict_proba(X_test)[:,0]

### Evaluating Results

In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

### Checking for over and underfitting

In [ ]:
y_pred_train = logreg.predict(X_train)

y_pred_train

print('Training set score: {:.4f}'.format(logreg.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(logreg.score(X_test, y_test)))